<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/rnn/rnn_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networks

In [2]:
import numpy as np

from numpy.random import randint

np.random.seed(42)

## Representing text as tokens

Let's define our dataset samples $x \in \mathrm{R}^d$, where $d$ is the feature space dimension.

With time sequences our data can be represented as $x \in \mathrm{R}^{t \, \times \, d}$, where $t$ is the sequence length. 
This emphasises sequence dependence and that the samples along the sequence are not independent and identically distributed (i.i.d.).
We will model functions as $\mathrm{R}^{t \, \times \, d} \rightarrow \mathrm{R}^c$, where $c$ is the amount of classes in the output.

There are several ways to represent sequences. With text, the challenge is how to represent a word as a feature vector in $d$ dimensions, as we are required to represent text with decimal numbers in order to apply neural networks to it.

Initially, we will use a simple one-hot encoding but for categorical variables that can take on many values (e.g. words in the English language).

### One-hot encoding over vocabulary

One way to represent a fixed amount of words is by making a one-hot encoded vector, which consists of 0s in all cells with the exception of a single 1 in a cell used uniquely to identify each word.

| vocabulary    | one-hot encoded vector   |
| ------------- |--------------------------|
| Paris         | $= [1, 0, 0, \ldots, 0]$ |
| Rome          | $= [0, 1, 0, \ldots, 0]$ |
| Copenhagen    | $= [0, 0, 1, \ldots, 0]$ |

Representing a large vocabulary with one-hot encodings often becomes inefficient because of the size of each sparse vector.
To overcome this challenge it is common practice to truncate the vocabulary to contain the $k$ most used words and represent the rest with a special symbol, $\mathtt{UNK}$, to define unknown/unimportant words.
This often causes entities such as names to be represented with $\mathtt{UNK}$ because they are rare.

Consider the following text
> I love the corny jokes in Spielberg's new movie.

where an example result would be similar to
> I love the corny jokes in $\mathtt{UNK}$'s new movie.

### Generating a dataset

We generate sequences of the form:

`a b EOS`,

`a a b b EOS`,

`a a a a a b b b b b EOS`

where `EOS` is a special character denoting the end of a sequence. The task is to predict the next token $t_n$, i.e. `a`, `b`, `EOS` or the unknown token `UNK` given a sequence of tokens $\{ t_{1}, t_{2}, \dots , t_{n-1}\}$, and we are to process sequences in a sequential manner. As such, the network will need to learn that e.g. 5 `b`s and an `EOS` token will be preceded by 5 `a`s.

In [7]:
def generate_dataset(num_sequences=2**8):
  """
  Generated a number of sequences as out dataset.
  """
  generate_random_token = lambda num_tokens: (
      f"{'a' * num_tokens}{'b' * num_tokens}<EOS>"
  )
  return [generate_random_token(randint(1, 12)) for _ in range(num_sequences)]


In [10]:
sequences = generate_dataset()
sequences[:5]

['aaaaaaaabbbbbbbb<EOS>',
 'ab<EOS>',
 'aaaaaaaabbbbbbbb<EOS>',
 'aaaabbbb<EOS>',
 'aaaaaaaaaaabbbbbbbbbbb<EOS>']